In [2]:
import numpy as np
import gurobipy as gp

## Question 1:
Max is in a pie eating contest that lasts 1 hour. Each torte that he eats takes 2 minutes to eat. Each apple pie that he eats takes 3 minutes. He receives 4 points for each torte and 5 points for each pie. Find the number of tortes and apple pies Max should eat to get the most points. Solve the problem using the graphical method.

Next, let’s see what happens if he would like to stick to his preference of eating at least as many pies as tortes. That is; the number of pies he eats should be greater than or equal to the number of tortes.

By how many points does this constraint decrease Max’s total points? (this is the part you will submit to the canvas answer.)

## Solution:
Objective Function -> Maximise 4T + 5P
Constraints:
1. 2T + 3P <= 60
2. T>=0, P>=0
Scenario 2:
Include P-T>=0

From the Graphical method:
Case 1 : We get optimal value when Max eats 30 Tortes and 0 pies

Case 2 : When he needs to eat Pie more than the number of tortes, we get Pie value as 12 and Tortes value as 12

## Question 2 : 
A farmer in Iowa owns 450 acres of land. He is going to plant each acre with wheat or corn. Each acre planted with wheat (corn) yields $2,000 ($3,000) profit, requires three (two) workers, and requires two (four) tons of fertilizer. There are currently 1,000 workers and 1,200 tons of fertilizer available.
Formulate and solve this problem using gurobi.

Next we want to see What happens to the decision variables and the total profit when the availability of fertilizer varies from 200 tons to 2200 tons in 100-ton increments.

At what level of fertilizer does the farmer discontinue producing wheat?  That is, what is the smallest level of available fertilizer that results in no wheat being produced?  Your answer should be one of the 100 ton incremental numbers, like 600 or 1300 or...

## Solution
Objective Function -> Maximise 2000W + 3000C
Constraints:
W + C <= 450
3W + 2C <=1000
2W + 4C <=1200

In [2]:
obj = np.array([2000,3000]) # objective vector
A = np.zeros((3,2)) # initialize constraint matrix
A[0,:] = [3,2] # workers constraint
A[1,:] = [2,4] # fertilizer constraint
A[2,:] = [1,1]
b = np.array([1000,1200,450]) # limits on production, storage, and demand
sense = np.array(['<','<','<']) # all constraints are less than or equal constraints

In [3]:
ojModel = gp.Model() # initialize an empty model

ojModX = ojModel.addMVar(2) # tell the model how many variables there are
# must define the variables before adding constraints because variables go into the constraints
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) # add the constraints to the model
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) # add the objective to the model...we'll talk about the None and the 0

ojModel.Params.OutputFlag = 0 # tell gurobi to shut up!!

Academic license - for non-commercial use only - expires 2022-09-15
Using license file /Users/Kaushik Kumaran/gurobi.lic


In [4]:
ojModel.optimize() 

In [5]:
ojModel.objVal

1000000.0

In [6]:
ojModX.x

array([200., 200.])

In [7]:
# Scenario 2 : Vary the amount of fertilizer from 200 to 2000
for i in range(200,2100,100):
    b = np.array([1000,i,450]) 
    ojModel = gp.Model() # initialize an empty model
    ojModX = ojModel.addMVar(2) # tell the model how many variables there are
    ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) # add the constraints to the model
    ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 
    ojModel.Params.OutputFlag = 0
    ojModel.optimize() 
    print(i)
    print(ojModel.objVal)
    print(ojModX.x)

200
200000.0
[100.   0.]
300
300000.0
[150.   0.]
400
400000.0
[200.   0.]
500
500000.0
[250.   0.]
600
600000.0
[300.   0.]
700
687500.0
[325.   12.5]
800
750000.0
[300.  50.]
900
812500.0
[275.   87.5]
1000
875000.0
[250. 125.]
1100
937500.0
[225.  162.5]
1200
1000000.0
[200. 200.]
1300
1062500.0
[175.  237.5]
1400
1125000.0
[150. 275.]
1500
1187500.0
[125.  312.5]
1600
1250000.0
[100. 350.]
1700
1300000.0
[ 50. 400.]
1800
1350000.0
[  0. 450.]
1900
1350000.0
[  0. 450.]
2000
1350000.0
[  0. 450.]


Thus the value of the fertilizer needs to be 1800 so that no wheat is produced

## Question 3 :
Star Oil Company is considering five different investment opportunities. The table below gives the required cash outflows and net present values in millions of dollars.
Star Oil has 40 million available for investment now (time 0); it estimates that one year from now (time 1) $20 million will be available for investment. Star Oil may purchase any fraction of each investment, but no more than 100% of each opportunity. In this case, the cash outflows and NPV are adjusted accordingly.
For example, if Star Oil purchases one-fifth of investment 3, then a cash outflow of 1/5 × 5 = 1 million dollars would be required at time 0, and a cash outflow of 1/5 × 5 = 1 million would be required at time 1. The one-fifth share of investment three would yield an NPV of 1/5 ∗ 16 = 3.2 million dollars. Star Oil wants to maximize the NPV that can be obtained by investing in investments 1-5. Formulate an LP that will help achieve this goal. Assume that any funds leftover at time 0 cannot be used at time 1.
What percentage of opportunity 3 should be Star Oil invest in?  Answer in decimals, so if your answer is 54%, you should input 0.54.  Round 2 to decimal places

## Solution : 
Decision Variables - a,b,c,d,e
Objective Function -> Maximise 13a + 16b + 16c + 14d + 39e
Subject to constrains:
11a + 53b + 5c + 5d + 29e <= 40
3a + 6b + 5c + d + 34e <= 20
a<=1
b<=1
c<=1
d<=1
e<=1

In [9]:
obj = np.array([13,16,16,14,39]) 
A = np.zeros((7,5)) 
A[0,:] = [11,53,5,5,29] 
A[1,:] = [3,6,5,1,34]
A[2,:] = [1,0,0,0,0]
A[3,:] = [0,1,0,0,0]
A[4,:] = [0,0,1,0,0]
A[5,:] = [0,0,0,1,0]
A[6,:] = [0,0,0,0,1]
b = np.array([40,20,1,1,1,1,1]) 
sense = np.array(['<','<','<','<','<','<','<'])

ojModel = gp.Model()

ojModX = ojModel.addMVar(5) 

ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

ojModel.Params.OutputFlag = 0

ojModel.optimize() 
print(ojModel.objVal)
print(ojModX.x)

57.449017199017206
[1.         0.20085995 1.         1.         0.28808354]


## Question 4 : 
The goal of this problem is to select a set of foods that will satisfy a set of daily nutritional requirement at minimum cost. Suppose there are three foods available, corn, milk, and bread. There are restrictions on the number of calories (between 2000 and 2250) and the amount of Vitamin A (between 5000 and 50,000) that can be eaten. The table below shows, for each food, the cost per serving, the amount of Vitamin A per serving, and the number of calories per serving. Also, the maximum number of servings for each food is 10.

How many servings of corn should you eat?  Round to 2 decimal places.

In [3]:
from gurobipy import GRB
obj = np.array([0.18,0.23,0.05])
A = np.zeros((7,3))
A[0,:] = [72,121,65]
A[1,:] = [72,121,65]
A[2,:] = [107,500,0]
A[3,:] = [107,500,0]
A[4,:] = [1,0,0]
A[5,:] = [0,1,0]
A[6,:] = [0,0,1]
b = np.array([2000,2250,5000,50000,10,10,10]) 
sense = np.array(['>','<','>','<','<','<','<']) 

ojModel = gp.Model()

ojModX = ojModel.addMVar(3) 
ojModCon = ojModel.addMConstrs(A, ojModX, sense, b) 
ojModel.setMObjective(None,obj,0,sense=gp.GRB.MINIMIZE)

ojModel.Params.OutputFlag = 0
ojModel.optimize() 
if ojModel.status == GRB.OPTIMAL: 
    print('Optimal objective: %g' % ojModel.objVal)
    print(ojModX.x)
elif ojModel.status == GRB.INF_OR_UNBD:
    print('Model is infeasible or unbounded')
elif ojModel.status == GRB.INFEASIBLE: 
        print('Model is infeasible')  
elif model.status == GRB.UNBOUNDED:    
    print('Model is unbounded')   
else:  
    print('Optimization ended with status %d' % ojModel.status) 


Academic license - for non-commercial use only - expires 2022-09-15
Using license file /Users/Kaushik Kumaran/gurobi.lic
Optimal objective: 3.15
[ 1.94444444 10.         10.        ]


## Question 5 
Paper and wood products companies need to define cutting schedules that will maximize the total wood yield of their forests over some planning period. Suppose that a firm with control of 2 forest units wants to identify the best cutting schedule over a planning horizon of 3 years. Forest unit 1 has a total acreage of 2 and unit 2 has a total of 3 acres. The studies that the company has undertaken predict that each acre in unit 1(2) will have 1, 1.3, 1.4 (1, 1.2, 1.6) tons of woods per acre available for harvesting in year 1, 2, 3 respectively. Based on its prediction of economic conditions, the company believes that it should harvest at least 1.2, 1.5, 2 tons of wood in year 1, 2, 3 separately. Due to the availability of equipment and personnel, the company can harvest at most 2, 2, 3 tons of wood in year 1, 2, 3. Find the company’s best cutting strategy that maximizes the total weights of wood. Here discounting of the time value should not be considered.  If some fraction of a forest unit is cut down in year 1, that part of the forest cannot be cut again for the remaining 2 years.  Similarly if some fraction of the forest unit is cut down in year 2 it cannot be cut in year 3.
In year 3, how many acres of forest unit 2 should be cut down?  Round to 2 decimal places.

In [3]:
obj = np.array([1,1.3,1.4,1,1.2,1.6]) # objective vector
A = np.zeros((8,6)) # initialize constraint matrix
A[0,:] = [1,0,0,1,0,0] # year 1 maximum wood constraint
A[1,:] = [1,0,0,1,0,0] # year 1 minimum wood constraint
A[2,:] = [0,1.3,0,0,1.2,0] # year 2 maximum wood constraint
A[3,:] = [0,1.3,0,0,1.2,0] # year 2 minimum wood constraint
A[4,:] = [0,0,1.4,0,0,1.6] # year 3 maximum wood constraint
A[5,:] = [0,0,1.4,0,0,1.6] # year 3 minimum wood constraint
A[6,:] = [1,1,1,0,0,0] # forest unit 1 constraint
A[7,:] = [0,0,0,1,1,1] # forest unit 2 constraint
b = np.array([2,1.2,2,1.5,3,2,2,3]) # limits on constraints
sense = np.array(['<','>','<','>','<','>','<','<'])


fmodel = gp.Model()
fmModX = fmodel.addMVar(6) 
fmModCon = fmodel.addMConstrs(A, fmModX, sense, b) 
fmodel.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE) 
fmodel.Params.OutputFlag = 0 
fmodel.optimize()




print("In year 3, acres of unit 2 to be cut down: ",round(fmModX.x[5],2))

Academic license - for non-commercial use only - expires 2022-09-15
Using license file /Users/Kaushik Kumaran/gurobi.lic
In year 3, acres of unit 2 to be cut down:  1.88
